# Evaluation
### 1. Export Model
Before the Model can be evaluated properly, it has to be exported from the most recent checkpoint. This is done by the `exporter_main_v2.py` script of the TensorFlow object detection API

In [ ]:
exporter_main_v2_path = '/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/scripts/exporter_main_v2.py'
trained_checkpoint_dir_path = '/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/06_models/models/ssd_resnet101_1024/v2/checkpoint/'
pipeline_config_path = '/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/06_models/models/ssd_resnet101_1024/v2/pipeline.config'
output_directory_path = '/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/06_models/models/ssd_resnet101_1024/v2/'

In [ ]:
# Limit this notebook to one GPU in order to preseve resources for other users.
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"


In [ ]:
!python {exporter_main_v2_path} \
    --trained_checkpoint_dir {trained_checkpoint_dir_path} \
    --output_directory {output_directory_path} \
    --pipeline_config_path {pipeline_config_path}

## 2. Recap of the Model Training

### Model Information
We trained a SSD (Single Stage Detector) model using a Resnet101 Backbone and a 1024x1024 input layer on quatered input images (resulting in a size of 1280x1280) for the input images. The images thus had to be slightly scales during model training. The model is pretrained on COCO-2017 using the checkpoints provided in the TensorFlow model Zoo. We use a batch size of four on four GPUs, a learning rate of 0.0025 with momentum optimizer and a toal of 15000 steps (=trained Batches).

### Training Progress
Before we actually test the final model on single images and the test dataset, let's first discuss what information can be obtained from the TensorBoard tracking information during model training.

In [ ]:
# Run TensorBoard in notebook
%load_ext tensorboard
%tensorboard --logdir /home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/06_models/models/ssd_resnet101_1024/v2

### Training Progress Analysis
A few interesting things can be told from the training graphs. First, lets explain some basics. The orange graphs show the metrics calculated from the performance on the train dataset. The blue graphs are on the eval dataset. It is clear that all the loss values are much lower on the training set than on the validation set, which suggests bad generalization capabilitie of the model, but we will have to look into that later. Interesting are the graphs depicting the different components of the COCO detection metrics (mAP) which was introduced in the training notebook. It is clear that the performance on the validation set reaches a maximum after about 15000 training steps after which it drops again, suggesting overfitting. It can also be observed that the filter for large objects that is provided by the COCO detection API is constantly at -1 mAP which means that the dataset does not contain any large objects. This is in line with the findings in the EDA notebook. 


## 2. Test trained model on test images

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display
from tqdm.notebook import tqdm_notebook

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

## Load Model
First, the model, that we previously generated has to be imported into tensorflow.

In [ ]:
print('Loading model...', end='')
start_time = time.time()

model = tf.saved_model.load(f'{output_directory_path}/saved_model')

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: a file path (this can be local or on colossus)

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

### Run Inferece on Entire Test-set
We now infere results on the test set and calculate the coco-metrics.

In [ ]:
def run_inference_aggregate_detections(model, image_path):
    model_fn = model.signatures['serving_default']
    results = []

    for image_name in glob.glob(image_path):
        # img_pillow = Image.open(image_name)
        # img_numpy = np.array(img_pillow.getdata()).reshape(img_pillow.size[0], img_pillow.size[1], 3).astype(np.uint8)
        img_numpy = load_image_into_numpy_array(image_name)

        img_tensor = tf.convert_to_tensor(img_numpy)

        img_tensor = img_tensor[tf.newaxis,...]

        output_dict = model_fn(img_tensor)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
        num_detections = int(output_dict.pop('num_detections'))
        output_dict = {key:value[0, :num_detections].numpy() 
                    for key,value in output_dict.items()}

        results.append(output_dict)


    return results


In [ ]:
img_path = '/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/04_feature/images/test/*jpg'

results = run_inference_aggregate_detections(model, img_path)

In [ ]:
from object_detection.core import standard_fields as fields

In [ ]:
## import ground truth
import pandas as pd
from ast import literal_eval

def import_ground_truth(annotations_file_path):

    annotations = pd.read_csv(annotations_file_path)

    def expand_bbox_coordinates(row: pd.Series):
        bbox_as_tuple = literal_eval(row["bounds"])
        row["bbox_x1"] = bbox_as_tuple[0]
        row["bbox_y1"] = bbox_as_tuple[1]
        row["bbox_x2"] = bbox_as_tuple[2]
        row["bbox_y2"] = bbox_as_tuple[3]

        return row

    annotations = annotations.apply(expand_bbox_coordinates, 1)

    image_names = annotations.image_id.unique()
    annotations_list = []

    for image_name in image_names:
        bboxes = []

        annotations_filtered = annotations[
            annotations["image_id"] == image_name
        ]
        xmins = annotations_filtered["bbox_x1"].values
        xmaxs = annotations_filtered["bbox_x2"].values
        ymins = annotations_filtered["bbox_y1"].values
        ymaxs = annotations_filtered["bbox_y2"].values

        for i in range(len(xmins)):
            bboxes.append([ymins[i], xmins[i], ymaxs[i], xmaxs[i]])

        bboxes = np.array(bboxes, dtype='float32')
        
        annotations_list.append(
            {fields.InputDataFields.groundtruth_boxes: bboxes, fields.InputDataFields.groundtruth_classes: np.array([1 for _ in range(len(bboxes))], dtype='int64')}
        )

    return image_names, annotations_list

In [ ]:
image_names, annotations_list = import_ground_truth("../data/04_feature/annotations/test.csv")

In [ ]:
def process_results(results, image_size):
    results_new = []

    for result in results:
        results_new.append(
            {
                fields.DetectionResultFields.detection_boxes: result["detection_boxes"]*image_size,
                fields.DetectionResultFields.detection_scores: result["detection_scores"],
                fields.DetectionResultFields.detection_classes: result["detection_classes"],
            }
        )

    return results_new

In [ ]:
processed_results = process_results(results, 1280)

In [ ]:
image_list = [
    filename.split("/")[-1].split(".")[0]
    for filename in glob.glob(img_path)
]

In [ ]:
from object_detection.metrics.coco_evaluation import CocoDetectionEvaluator

cde = CocoDetectionEvaluator([{"id": 1, "name": "OST"}])

In [ ]:
for i in range(len(image_list)):
    cde.add_single_ground_truth_image_info(
        image_names[i], annotations_list[i]
    )
    

In [ ]:
for i in range(len(image_list)):
    cde.add_single_detected_image_info(
            image_list[i], processed_results[i]
        )

In [ ]:
scores_test = cde.evaluate()

### Single Image Intferece
Alternatively, we can display the boxes on top of the images. Code copied from Zengerle & Göhl.

In [ ]:
labelmap_path = '/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/04_feature/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
test_images_path = '/home/dammeier@ab.ba.ba-ravensburg.de/dev/oil-storage-detection/data/04_feature/images/test/*jpg'

image_paths = glob.glob(test_images_path)

image_path = image_paths[3]

#for image_path in glob.glob(test_images_path):
image_np = load_image_into_numpy_array(image_path)
output_dict = run_inference_for_single_image(model, image_np)
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    line_thickness=8)
display(Image.fromarray(image_np))